In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def webScrape(links):
  unique_urls = set()
  for link in links:
    response = requests.get(link)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    buttons = soup.find_all('a')
    url = {button['href'] for button in buttons if 'href' in button.attrs}
    unique_urls.update(url)

    for link in url:
        print(link)
    print(f"Total Links: {len(unique_urls)}")
  return unique_urls

In [3]:
url = ['https://results.eci.gov.in/']
urls = webScrape(url)

https://apps.apple.com/in/app/voter-helpline/id1456535004
https://results.eci.gov.in/AcResultGenJune2024/index.htm
https://results.eci.gov.in/PcResultGenJune2024/index.htm
https://play.google.com/store/apps/details?id=com.eci.citizen
https://results.eci.gov.in/AcResultGen2ndJune2024/index.htm
index.htm
https://results.eci.gov.in/AcResultByeJune2024/
Total Links: 7


In [5]:
filtered_urls = [url for url in urls if url.endswith('/index.htm')]
print(filtered_urls)

['https://results.eci.gov.in/AcResultGen2ndJune2024/index.htm', 'https://results.eci.gov.in/AcResultGenJune2024/index.htm', 'https://results.eci.gov.in/PcResultGenJune2024/index.htm']


In [14]:
import requests
from bs4 import BeautifulSoup

# List of URLs to traverse
filtered_urls = [
    'https://results.eci.gov.in/AcResultGen2ndJune2024/index.htm',
    'https://results.eci.gov.in/AcResultGenJune2024/index.htm',
    'https://results.eci.gov.in/PcResultGenJune2024/index.htm'
]

# Function to fetch and parse data from a URL
def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        page_content = response.content
        soup = BeautifulSoup(page_content, 'html.parser')
        return soup
    else:
        print(f"Failed to retrieve {url}")
        return None

# Function to categorize tables
def categorize_tables(soup, url):
    class_table = []
    non_class_table = []
    
    tables = soup.find_all('table')
    for table in tables:
        if 'table' in table.get('class', []):
            class_table.append(url)  # Append URL instead of table object
        else:
            non_class_table.append(table)
    
    return class_table, non_class_table

# Function to extract and print data from a list of tables
def extract_data(tables, category):
    print(f"\nExtracting data from {category} tables...")
    for table in tables:
        print(table)  # Print the URL instead of the table object

# Traverse each URL
for url in filtered_urls:
    print(f"Fetching data from {url}")
    soup = fetch_data(url)
    if soup:
        class_tables, non_class_tables = categorize_tables(soup, url)
        
        # Print the URLs of class tables
        print(f"Class tables found in {url}:")
        print()
        
        for table_url in class_tables:
            print(table_url)


Fetching data from https://results.eci.gov.in/AcResultGen2ndJune2024/index.htm
Class tables found in https://results.eci.gov.in/AcResultGen2ndJune2024/index.htm:

Fetching data from https://results.eci.gov.in/AcResultGenJune2024/index.htm
Class tables found in https://results.eci.gov.in/AcResultGenJune2024/index.htm:

Fetching data from https://results.eci.gov.in/PcResultGenJune2024/index.htm
Class tables found in https://results.eci.gov.in/PcResultGenJune2024/index.htm:

https://results.eci.gov.in/PcResultGenJune2024/index.htm


In [13]:
print(class_tables)

['https://results.eci.gov.in/PcResultGenJune2024/index.htm']
